In [1]:
from moviepy.editor import VideoFileClip
import numpy as np
import pandas as pd
import scipy.stats
import pandas as pd
import glob
import json
import librosa
import soundfile as sf
import io
import os

In [2]:
# video_clip = VideoFileClip("train_sample_videos/aagfhgtpmv.mp4")
# audio = video_clip.audio

# # Step 2: Convert the audio into an array of samples
# fps = audio.fps  # frames per second (sampling rate)
# audio_samples = np.array(list(audio.iter_frames(fps=fps, dtype="float32"))).flatten()

# # Step 3: Use an in-memory buffer to store the audio as a .wav file
# buffer = io.BytesIO()
# sf.write(buffer, audio_samples, fps, format='wav')  # Store audio in buffer as .wav
# buffer.seek(0)  # Reset buffer to the beginning

# # Step 4: Load audio from buffer using librosa
# x, sr = librosa.load(buffer, sr=None)

Statistical Features  
A first easy step is to compute the mean, standard deviation, minimum, maximum, median and quartiles of the frequencies of each signal. This can be done using Numpy and it always brings value to our feature extraction.

In [3]:
# freqs = np.fft.fftfreq(x.size)

# def describe_freq(freqs):
#     mean = np.mean(freqs)
#     std = np.std(freqs)
#     maxv = np.amax(freqs)
#     minv = np.amin(freqs)
#     median = np.median(freqs)
#     skew = scipy.stats.skew(freqs)
#     kurt = scipy.stats.kurtosis(freqs)
#     q1 = np.quantile(freqs, 0.25)
#     q3 = np.quantile(freqs, 0.75)
#     mode = scipy.stats.mode(freqs)[0][0]
#     iqr = scipy.stats.iqr(freqs)

#     return [mean, std, maxv, minv, median, skew, kurt, q1, q3, mode, iqr]

In [4]:
# mfcc = list(np.mean(librosa.feature.mfcc(y=x, sr=sr), axis=1))
# print(len(mfcc))

In [5]:
def get_features(x, sr):
    rmse = np.mean(librosa.feature.rms(y=x)[0])
    zcr = np.mean(librosa.feature.zero_crossing_rate(x)[0])
    tempo = librosa.beat.tempo(y=x, sr=sr)[0]
    mfcc = list(np.mean(librosa.feature.mfcc(y=x, sr=sr), axis=1))
    spec_cen = np.mean(librosa.feature.spectral_centroid(y=x, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=x, sr=sr))
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=x, sr=sr))
    spectral_flatness = np.mean(librosa.feature.spectral_flatness(y=x))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=x, sr=sr))
    features = [rmse, zcr, tempo, spec_cen, spectral_bandwidth, spectral_contrast, spectral_flatness, spectral_rolloff]
    return features + mfcc

In [6]:
# column_names = ['FileName', 'rmse', 'zcr', 'tempo', 'spectral_centroid', 'spectral_bandwidth', 'spectral_contrast', 'spectral_flatness', 'spectral_rolloff'] + ['mfcc' + str(i) for i in range(1, 21)] + ['label']
# file_names = glob.glob("train_sample_videos/*.mp4")
# df = pd.DataFrame(columns=column_names)

In [7]:
# file = "train_sample_videos/aagfhgtpmv.mp4"

In [8]:
real_audio_dir = r"REAL"
fake_audio_dir = r"FAKE"

In [9]:
def extract_features(file_path):
    try:
        # Load the video file
        video_clip = VideoFileClip(file_path)
        audio = video_clip.audio
        fps = audio.fps
        audio_samples = np.array(
            list(audio.iter_frames(fps=fps, dtype="float32"))
        ).flatten()
        buffer = io.BytesIO()
        sf.write(buffer, audio_samples, fps, format="wav")
        buffer.seek(0)
        x, sr = librosa.load(buffer, sr=None)
        features = get_features(x, sr)

        return features

    except Exception as e:
        print(f"Error encountered while parsing file: {file_path}, {e}")
        return None


def load_data(real_dir, fake_dir):
    """Load and process audio files from real and fake directories."""
    data = []

    # Process real audio files
    for file_name in os.listdir(real_dir):
        file_path = os.path.join(real_dir, file_name)
        features = extract_features(file_path)
        if features is not None:
            features["label"] = 0  # 0 for REAL
            data.append(features)

    # Process fake audio files
    for file_name in os.listdir(fake_dir):
        file_path = os.path.join(fake_dir, file_name)
        features = extract_features(file_path)
        if features is not None:
            features["label"] = 1  # 1 for FAKE
            data.append(features)

    df = pd.DataFrame(data)

    columns = (
        [
            "rmse",
            "zcr",
            "tempo",
            "spectral_centroid",
            "spectral_bandwidth",
            "spectral_contrast",
            "spectral_flatness",
            "spectral_rolloff",
        ]
        + [f"mfcc{i}" for i in range(1, 21)]
        + ["label"]
    )

    return df[columns]

In [10]:
df = pd.DataFrame(
    columns=[
        "rmse",
        "zcr",
        "tempo",
        "spectral_centroid",
        "spectral_bandwidth",
        "spectral_contrast",
        "spectral_flatness",
        "spectral_rolloff",
        "mfcc1",
        "mfcc2",
        "mfcc3",
        "mfcc4",
        "mfcc5",
        "mfcc6",
        "mfcc7",
        "mfcc8",
        "mfcc9",
        "mfcc10",
        "mfcc11",
        "mfcc12",
        "mfcc13",
        "mfcc14",
        "mfcc15",
        "mfcc16",
        "mfcc17",
        "mfcc18",
        "mfcc19",
        "mfcc20",
        "label",]
)

In [11]:
df = load_data(real_audio_dir, fake_audio_dir)

AttributeError: 'tuple' object has no attribute 'head'

In [10]:
# for file in file_names:
#     clean_file = file.split("/")[-1]
#     video_clip = VideoFileClip(file)
#     audio = video_clip.audio
#     fps = audio.fps
#     audio_samples = np.array(list(audio.iter_frames(fps=fps, dtype="float32"))).flatten()
#     buffer = io.BytesIO()
#     sf.write(buffer, audio_samples, fps, format='wav')
#     buffer.seek(0)
#     x, sr = librosa.load(buffer, sr=None)
#     label = json.load(open("train_sample_videos/metadata.json"))[clean_file]['label']
#     new_row = pd.DataFrame([[clean_file] + get_features(x, sr) + [label]], columns=column_names)
#     df = pd.concat([df, new_row], ignore_index=True)

In [14]:
df.to_csv("full_features.csv", index=False)